# Imports and options

In [2]:
import numpy as np
import pandas as pd
import datatable as dt
from googleapiclient.discovery import build
from datetime import timedelta

# Display options:
pd.set_option("display.width", 1200)
pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 300)

# Youtube Data API

In [46]:
yt = build(
    serviceName = "youtube",
    version = "v3",
    developerKey = "" # my key
)

# Channels sample

In [47]:
# * the API only returns 20.000 videos by channel
# Ids from (content = "https://www.youtube.com/channel/...) and titles:
channels = {
    "UCYO_jab_esuFRV4b17AJtAw": "3Blue1Brown",
    "UCNKcMBYP_-18FLgk4BYGtfw": "AWE me",
    "UCiDJtJKMICpb9B1qf7qjEOA": "Adam Savage’s Tested",
    "UC32z4mtyiq02Ge-XWy78ibw": "Aero Por Trás da Aviação",
    "UCWizIdwZdmr43zfxlCktmNw": "Alec Steele",
    "UCvFGf8HZGZWFzpcDCqb3Lhw": "All Things Secured",
    "UCaRMivfyupj3ucUyJbZbCNg": "Anglo-Link",
    "UCwg6_F2hDHYrqbNSGjmar4w": "Animalogic",
    "UCi5iiEyLwSLvlqnMi02u5gQ": "Ask a Mortician",
    "UCGYBY4KaFYmkEKAGLL07BXw": "AstroTubers",
    "UC-9b7aDP6ZN0coj9-xFnrtw": "Astrum",
    "UCSTlOTcyUmzvhQi6F8lFi5w": "Atila Iamarino",
    "UCLXl1V6n82Dyg1VhVgSL0nw": "Aviões e Músicas com Lito Sousa",
    "UCGlmclFVnJCFQ_VO4kTgTig": "Azusa Barbie",
    "UCCj956IF62FbT7Gouszaj9w": "BBC",
    "UCvZe6ZCbF9xgbbbdkiodPKQ": "Baumgartner Restoration",
    "UCxcnsr1R5Ge_fbTu5ajt8DQ": "Bob Ross",
    "UCQRPDZMSwXFEDS67uc7kIdg": "Buenas Ideias",
    "UCcyq283he07B7_KUX07mmtA": "Business Insider",
    "UCvJJ_dzjViJCoLf5uKUTwoA": "CNBC",
    "UCWq1xltHB2fDe6YkYoOrryg": "Canal do Schwarza",
    "UCEOXxzW2vU0P-0THehuIIeg": "Captain Disillusion",
    "UCwnKziETDbHJtx78nIkfYug": "CaspianReport",
    "UCR1-7g_y2YcYlh1W9y_1LUg": "Chaves Estranho",
    "UC04KsGq3npibMCE9Td3mVDg": "Cheddar",
    "UC6HDXr-sNPnWLF_Q-y3KduA": "Chess Talk",
    "UCpVm7bg6pXKo1Pr6k5kxG9A": "Classic Mr Bean",
    "UCvmijL-eepDVHYSJHDY3d6w": "Cole and Marmalade",
    "UC9-y-6csu5WGm29I7JiwpnA": "Computerphile",
    "UCyAMV63OY0DRbKk6gGv6wJg": "Cradle Of Filth",
    "UCe_vXdMrHHseZ_esYUskSBw": "CrazyRussianHacker",
    "UC726J5A0LLFRxQ0SZqr2mYQ": "Curious Droid",
    "UCFuIUoyHB12qpYa8Jpxoxow": "Código Fonte TV",
    "UCW39zufHfsuGgpLviKh297Q": "DW Documentary",
    "UCd-swDW3HCs4LTZhq8Cf7Fg": "DW Euromaxx",
    "UCQIcXQ2n0sa-7CD0NtqnrrA": "Darko Audio",
    "UCa37IMrH8BGS_pO6CKbTL-A": "Dinosaurs",
    "UCEf5U1dB5a2e2S-XUlnhxSA": "Diolinux",
    "UC_5niPa-d35gg88HaS7RrIw": "Disney",
    "UC1xwwLwm6WSMbUn_Tp597hQ": "Disney Parks",
    "UCKUNiU0D2u3yxNZZxBqa6bQ": "DutchPilotGirl",
    "UCVRrGAcUc7cblUzOhI1KfFg": "Earthling Ed",
    "UCIBaDdAbGlFDeS33shmlD0A": "European Space Agency, ESA",
    "UC6uKrU_WqJ1R2HMTY3LIx5Q": "Everyday Astronaut",
    "UCZ6JHFBaDUJ9wfo41HSII_w": "Fabio Chaves",
    "UC6zbvGt_jJVgw7-MWkVUC5A": "Fala Vegan",
    "UCxGiUjep8KiihvHldRousPA": "Fancy Fairy Wings & Things",
    "UCweCc7bSMX5J4jEH7HFImng": "GMHikaru",
    "UCuTaETsuCOkJ0H_GAztWt0Q": "Global Cycling Network",
    "UCeSRjhfeeqIgr--AcP9qhyg": "Guinness World Records",
    "UCLAcytNR3gdw44yzoSiKluA": "Hello Korea",
    "UC26YLK0OEbLB3TCYxGh8xVQ": "Huygens Optics",
    "UCcMDMoNu66_1Hwi5-MeiQgw": "Hydraulic Press Channel",
    "UC3uAjWoLZ4bSi6qI9SjALxA": "Imperial War Museums",
    "UCHJuQZuzapBh-CuhRYxIZrg": "Insider",
    "UCkAbeyJZqp6JFYfKy-98NjA": "Integrando Conhecimento",
    "UCJ8bjM5yQSkOP-_99n1zKkw": "Intensivo Pedagógico",
    "UCZ9l_6_f0PWRYXN5Y7Lcl2A": "Jacques Slade",
    "UCbbQalJ4OaC0oQ0AqRaOJ9g": "Jay Foreman",
    "UCkP2CvRubyU0MTZv_Qo712g": "Jessica in the Kitchen - Vegan Recipes",
    "UC-2YHgc363EdcusLIBbgxzg": "Joe Scott",
    "UCBbnbBWJtwsf0jLGUwX5Q3g": "Journey to the Microcosmos",
    "UCLQWhXC5aOZ_tQNloBg40BA": "Kirsten & Joerg",
    "UCuCuEKq1xuRA0dFQj1qg9-Q": "Knowledgia",
    "UCsXVk37bltHxD1rDPwtNM8Q": "Kurzgesagt – In a Nutshell",
    "UCNqNkZ7kKfqimqHkgbWMNYA": "Launch Pad Astronomy",
    "UCoNTMWgGuXtGPLv9UeJZwBw": "Living Big In A Tiny House",
    "UCm9K6rby98W8JigLoZOh6FQ": "LockPickingLawyer",
    "UCAcZNVPEQ8K7JhMDwOcf8Pg": "Loira na Estrada",
    "UCt2WVZXVrHaFgZ432s33fLw": "MAKE UP FOR EVER",
    "UCbK5Us4E-HsXw6fQ1PYUuog": "MW Informática",
    "UCXgxNzAgZ1GExhTW4X1mUrg": "Marc Rebillet",
    "UCBlXovStrlQkVA2xJEROUNg": "Marcel Vos",
    "UCr22xikWUK2yUW4YxOKXclQ": "MathTheBeautiful",
    "UC6nZXhOz_gHr4mlb5UNl9ug": "Matt Jordan",
    "UCQwFuQLnLocj5F7ZcmcuWYQ": "MetaBallStudios",
    "UCk5BcU1rOy6hepflk7_q_Pw": "Meteoro Brasil",
    "UCqONzeACDBaF6FfKjh7ndAQ": "Microsoft Flight Simulator",
    "UChHS4NI6U4XgCuYgsrygVCA": "Mintfaery",
    "UC1ZBQ-F-yktYD4m5AzM6pww": "Mustard",
    "UCLA_DiR1FfKNvjuUpBHmylQ": "NASA",
    "UCSuHzQ3GrHSzoBbwrIq3LLA": "Naomi Brockwell",
    "UCPZvqkyXoYDlzBClxpxVIzg": "Naomi Farr",
    "UCggHsHce2n3vvbJf_8YKrMA": "Nerdforge",
    "UCWOA1ZGywLbqmigxE4Qlvuw": "Netflix",
    "UCD57tGnYPW1twCohgIVkJpw": "Nox et Lux",
    "UCoxcjq-8xIDTYp3uz647V5A": "Numberphile",
    "UCroDJPcFCf6DBmHns6Xeb8g": "Nyma Tang",
    "UCjivwB8MrrGCMlIuoSdkrQg": "Nátaly Neri",
    "UCQ3JxE-NOyZaJ3m3qIkZbhA": "O Pimentinha",
    "UCK1XzxcKXB_v_dcw_tflC1A": "OBF",
    "UCtb2f3j_sB4Z4Ik2YJtbzsg": "OneikaTraveller",
    "UCwK-svlNEASA7g_smu1_d8g": "OnlyConnect Fan",
    "UCpLQXR116cLVUa1LRY8KS4w": "OwlKitty",
    "UCzR-rom72PHN9Zg7RML9EbA": "PBS Eons",
    "UC7_gcs09iThXybpVgjHZ_7g": "PBS Space Time",
    "UCddYq41_tZ1FnLlguLT6-Ow": "Parafernalha",
    "UCOVkkVaUP0Xop6IIjyk86QA": "Perception",
    "UCtESv1e7ntJaLJYKIO1FoYw": "Periodic Videos",
    "UC7DdEm33SyaTDtWYGO2CwdA": "Physics Girl",
    "UCQwHYU3ZbSLG6LOVwtvuozg": "Planeta Aves",
    "UCEWHPFNilsT0IfQfutVzsag": "Porta dos Fundos",
    "UCNJe8uQhM2G4jJFRWiM89Wg": "Potato Jet",
    "UClo-U5gvXPRy-4VqWEkkRDg": "Primitive Building",
    "UCKKJpBveT8vWVNfLQ-MvZMg": "Prof. André Azevedo da Fonseca",
    "UCXOzDw_X92bNqSCqw-NojiA": "Professor Julio Borbo",
    "UCwSxSJqGpSRpEsq5-YUbM8g": "Professor Noslen",
    "UCNzul4dnciIlDg8BAcn5-cQ": "Prowalk Tours",
    "UCvn_XCl_mgQmt3sD753zdJA": "Rachel's English",
    "UCR1IuLEqb6UEA_zQ81kwXfg": "Real Engineering",
    "UCP5tjEmvPItGyLhmjdwP7Ww": "RealLifeLore",
    "UCFJxE0l3cVYU4kHzi4qVEkw": "Rebecca Watson",
    "UCliwOcLaEfqkGJ7xq2ipq4w": "Reverse Engineering",
    "UC9Ep0Y4T5rvUuIfjKN1wqTw": "Riddle",
    "UCovtFObhY9NypXcyHxAS7-Q": "Roberto Blake",
    "UCZYTClx2T1of7BRZ86-8fow": "SciShow",
    "UCxzC4EngIsMrPmbm6Nxvb-A": "Scott Manley",
    "UC3KEoMzNz8eYnwBC34RaKCQ": "Simone Giertz",
    "UCT3wwJBh8fwePiNC63NlzCA": "Simple Flying",
    "UC6107grRI4m0o2-emgoDnAA": "SmarterEveryDay",
    "UC_Fk7hHbl7vv_7K8tYqJd5A": "SpaceToday",
    "UCtI0Hodo5o5dUb67FeUjDeA": "SpaceX",
    "UC00uG71I6iPyx15EX6i_GDA": "Star Wars Comics",
    "UCZ9jWH_8tJ-Nmaj8dSQdEYA": "Stefan Milo",
    "UCpQLC-evmUAon9BBpcW4kYg": "Steve Cutts",
    "UC6u6uY4VbvuNtU0BU7F9olw": "Studson Studio",
    "UCRXRbi80k0_vcIfgpOSerTg": "Subject Zero Science",
    "UCAuUUnT6oDeKwE6v1NGQxug": "TED",
    "UC7U72Ze-y8s4hvAmRvhXPgQ": "TecLab",
    "UCZdJE8KpuFm6NRafHTEIC-g": "Tempero Drag",
    "UCnUXq8mGmoHt0e6ItuTs10w": "That Pedal Show",
    "UC8uT9cgJorJPWu7ITLGo9Ww": "The 8-Bit Guy",
    "UC6n8I1UDTKP1IWjQMg6_TwA": "The B1M",
    "UCvpQ-l09fCVxJd3urZbxzHg": "The British Museum",
    "UCsQXQMAZTygXddj_I4Ud9xA": "The Green Witch",
    "UC42VsoDtra5hMiXZSsD6eGg": "The Modern Rogue",
    "UCUK0HBIBWgM2c4vsPhkYY4w": "The Slow Mo Guys",
    "UC2LVhJH_9cT2XKp0VAfsKOQ": "The Tim Traveller",
    "UCBa659QWEk1AI4Tg--mrJ2A": "Tom Scott",
    "UCN3aYbtQ7yCqk9DM56B0kEw": "Tomorrow's Build",
    "UClVbhSLxwws-KSsPKz135bw": "TopMovieClips",
    "UC29ju8bIPH5as8OGnQzwJyA": "Traversy Media",
    "UCgtq_tKnFFQyJ-7j0yr9utg": "Tyta Montrase",
    "UCSxFcMDGmwgDlSP9qmYVx1w": "TÁ NA MESA VEGG",
    "UCVEVuanoMK9tGclfWLghaKw": "Tá Querida",
    "UCv3fc4d_IhCK2g5Y_kjpvaw": "UM BOTÂNICO NO APARTAMENTO",
    "UCTlLFaGX4q2otO0ZVxLW0VQ": "Uma história a mais",
    "UC9fSZHEh6XsRpX-xJc6lT3A": "UrAvgConsumer",
    "UCbYFhcKSE2mWYB0yD_Qr_8A": "Urban Gardening",
    "UCc9M8_pF78tYblW8nMHXUqg": "VIEWGANAS",
    "UCp3iXxis9n_E_GfbE-_ksFw": "VeganBlackMetalChef",
    "UCHnyfMqiRRG1u-2MsSQLbXA": "Veritasium",
    "UCYulrOk4EbqL833oETw0bWQ": "Wanderlust Travel Videos",
    "UCs9MJ-TmN0-CLY2eNBHAltw": "Wanna Walk",
    "UCmVa-cbCpkd5Cd9Fr_4tCWg": "Your Dinosaurs Are Wrong",
    "UC9Ntx-EF3LzKY1nQ5rTUP2g": "cyriak",
    "UCtYKe7-XbaDjpUwcU5x0bLg": "neo",
    "UCVpankR4HtoAVtYnFDUieYA": "zefrank1"
 }

In [48]:
len(channels)

158

# Videos data

In [49]:
# Loop in the channels:
df_videos = pd.DataFrame(
    {
        "channel_title": pd.Series([], dtype = "str"),
        "channel_id": pd.Series([], dtype = "str"),
        "video_title": pd.Series([], dtype = "str"),
        "video_id": pd.Series([], dtype = "str"),
        "video_upload_date": pd.Series([], dtype = "str"),
        "views": pd.Series([], dtype = "int"),
        "likes": pd.Series([], dtype = "int"),
        "dislikes": pd.Series([], dtype = "int"),
        "comments": pd.Series([], dtype = "int")
    }
)
for chnl_ind, chnl_key in enumerate(channels):

    # Choose the channel:
    channel_id = chnl_key
    channel_title = channels[channel_id]

    print(channel_title)

    ### Extract the data from all the videos of the channel

    # Content details:
    content = yt.channels().list(
        id = channel_id,
        part = "contentDetails"
    ).execute()

    # Upload Id:
    upload_id = content["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # All the channel's videos:
    all_videos = []
    next_pg_token = None
    res = yt.playlistItems().list(
        playlistId = upload_id,
        maxResults = 50,
        part = "snippet",
        pageToken = next_pg_token
    ).execute()
    while True:
        res = yt.playlistItems().list(
            playlistId = upload_id,
            maxResults = 50,
            part = "snippet",
            pageToken = next_pg_token
        ).execute()
        all_videos += res["items"]
        next_pg_token = res.get("nextPageToken")
        if next_pg_token is None:
            break

    # Videos statistics:
    videos_ids = list(map(lambda x: x["snippet"]["resourceId"]["videoId"], all_videos))
    stats = []
    for i in range(0, len(videos_ids), 40):
        res = (yt).videos().list(
            id = ",".join(videos_ids[i:i + 40]),
            part = "statistics"
        ).execute()
        stats += res["items"]

    # Put it in a dataframe:
    video_id, views, likes, dislikes, comments = [], [], [], [], []
    for i in range(len(stats)):
        video_id += [stats[i]["id"]]
        stats_i = stats[i]["statistics"]
        views += [stats_i[k] if "viewCount" in stats_i.keys() else np.nan for k in ["viewCount"]]
        likes += [stats_i[k] if "likeCount" in stats_i.keys() else np.nan for k in ["likeCount"]]
        dislikes += [stats_i[k] if "dislikeCount" in stats_i.keys() else np.nan for k in ["dislikeCount"]]
        comments += [stats_i[k] if "commentCount" in stats_i.keys() else np.nan for k in ["commentCount"]]
    df_channel_videos = pd.DataFrame(
        {
            "channel_title": channel_title,
            "channel_id": channel_id,
            "video_title": list(map(lambda x: x["snippet"]["title"], all_videos)),
            "video_id": video_id,
            "video_upload_date": list(map(lambda x: x["snippet"]["publishedAt"], all_videos)),
            "views": views,
            "likes": likes,
            "dislikes": dislikes,
            "comments": comments        
        }
    )
    df_videos = pd.concat([df_videos, df_channel_videos])

3Blue1Brown
AWE me
Adam Savage’s Tested
Aero Por Trás da Aviação
Alec Steele
All Things Secured
Anglo-Link
Animalogic
Ask a Mortician
AstroTubers
Astrum
Atila Iamarino
Aviões e Músicas com Lito Sousa
Azusa Barbie
BBC
Baumgartner Restoration
Bob Ross
Buenas Ideias
Business Insider
CNBC
Canal do Schwarza
Captain Disillusion
CaspianReport
Chaves Estranho
Cheddar
Chess Talk
Classic Mr Bean
Cole and Marmalade
Computerphile
Cradle Of Filth
CrazyRussianHacker
Curious Droid
Código Fonte TV
DW Documentary
DW Euromaxx
Darko Audio
Dinosaurs
Diolinux
Disney
Disney Parks
DutchPilotGirl
Earthling Ed
European Space Agency, ESA
Everyday Astronaut
Fabio Chaves
Fala Vegan
Fancy Fairy Wings & Things
GMHikaru
Global Cycling Network
Guinness World Records
Hello Korea
Huygens Optics
Hydraulic Press Channel
Imperial War Museums
Insider
Integrando Conhecimento
Intensivo Pedagógico
Jacques Slade
Jay Foreman
Jessica in the Kitchen - Vegan Recipes
Joe Scott
Journey to the Microcosmos
Kirsten & Joerg
Knowledgia
K

# Data cleaning

## Change the data types

In [76]:
def toint64(x):
    return x.str.extract("(\d+)", expand = False).astype("float").astype("Int64")

df_videos["views"] = toint64(df_videos["views"])
df_videos["likes"] = toint64(df_videos["likes"])
df_videos["dislikes"] = toint64(df_videos["dislikes"])
df_videos["comments"] = toint64(df_videos["comments"])
df_videos["video_upload_date"] = pd.to_datetime(df_videos["video_upload_date"])

In [79]:
df_videos.dtypes

channel_title                     object
channel_id                        object
video_title                       object
video_id                          object
video_upload_date    datetime64[ns, UTC]
views                              Int64
likes                              Int64
dislikes                           Int64
comments                           Int64
dtype: object

## Remove videos with NaN or 0 in any variable

In [90]:
df_videos = df_videos.dropna(axis = 0)
df_videos = df_videos.loc[(df_videos["views"] > 0) & (df_videos["likes"] > 0) & (df_videos["dislikes"] > 0) & (df_videos["comments"] > 0)]
df_videos = df_videos.reset_index(drop = True)

In [91]:
df_videos

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments
0,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,A few of the best math explainers from this su...,F3Qixy-r_rQ,2021-10-23 18:11:23+00:00,502452,29457,107,768
1,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,How a Mandelbrot set arises from Newton’s work,LqbZpur38nw,2021-10-15 16:41:50+00:00,600077,26908,150,1237
2,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,Newton's Fractal (which Newton knew nothing ab...,-RdOwhmqP5s,2021-10-07 02:19:39+00:00,1235429,62901,294,2850
3,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,The Summer of Math Exposition,ojjzXyQCzso,2021-07-16 15:37:16+00:00,613493,29362,215,1722
4,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,A quick trick for computing eigenvalues | Chap...,e50Bj7jn9IQ,2021-05-07 19:01:16+00:00,427767,17155,145,1150
...,...,...,...,...,...,...,...,...,...
132280,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Art Hour,C6hpMftAIs0,2009-06-13 18:32:12+00:00,172809,2160,31,136
132281,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Hole,LPfcGXMpKds,2009-06-06 22:16:56+00:00,90894,1799,23,75
132282,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Affordable Button,h3lvSflNixI,2009-05-30 00:13:46+00:00,103229,1570,47,18
132283,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Outsource,zxIbQ6ZG4OI,2009-05-17 00:50:31+00:00,78267,1625,20,87


# New features

## Age in days (until the last upload date)

In [92]:
# Filter the wrong upload dates (coming from the API as being the request date) considering a margin of 2 days:
last_date = max(df_videos["video_upload_date"])
age = []
for i in range(df_videos.shape[0]):
    date_i = df_videos["video_upload_date"].iloc[i]
    diff_days = last_date - date_i
    if diff_days < timedelta(days = 2):
        age += [np.nan]
    else:
        age += [diff_days.days]
df_videos["age_days"] = age

df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [93]:
df_videos["age_days"].describe()

count    132204.000000
mean       1730.137908
std        1184.179437
min           2.000000
25%         814.000000
50%        1555.000000
75%        2381.000000
max        5743.000000
Name: age_days, dtype: float64

In [94]:
df_videos.sort_values("age_days").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days
86158,Meteoro Brasil,UCk5BcU1rOy6hepflk7_q_Pw,O QUE O BANQUEIRO DIZ QUANDO ACHA QUE VOCÊ NÃO...,7fD_d_VI738,2021-11-01 16:56:48+00:00,128665,22219,140,925,2.0
80203,Jacques Slade,UCZ9l_6_f0PWRYXN5Y7Lcl2A,Get Your Hands Up,FXLXBfmlaOk,2006-02-12 20:31:24+00:00,28237,207,9,108,5743.0


## Likes/dislikes

In [95]:
df_videos["likes_dislikes_ratio"] = df_videos["likes"]/df_videos["dislikes"]
df_videos["likes_dislikes_ratio"] = df_videos["likes_dislikes_ratio"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [96]:
df_videos["likes_dislikes_ratio"].describe()

count    132204.000000
mean         60.219593
std          77.113881
min           0.015103
25%          16.407448
50%          37.386403
75%          75.042981
max        2638.000000
Name: likes_dislikes_ratio, dtype: float64

In [97]:
df_videos.sort_values("likes_dislikes_ratio").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio
46864,Classic Mr Bean,UCpVm7bg6pXKo1Pr6k5kxG9A,American Gypsies | National Geographic,Zkz8F6xexbY,2012-07-02 21:43:01+00:00,267807,201,13309,737,3411.0,0.015103
128769,TÁ NA MESA VEGG,UCSxFcMDGmwgDlSP9qmYVx1w,"COMO ARMAZENAR FOLHAS, ORGANIZAR FEIRA + SAL T...",GGqRCQmtx-U,2021-05-21 01:16:50+00:00,15411,2638,1,261,167.0,2638.0


## Likes/views

In [98]:
df_videos["likes_views_ratio"] = df_videos["likes"]/df_videos["views"]
df_videos["likes_views_ratio"] = df_videos["likes_views_ratio"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [99]:
df_videos["likes_views_ratio"].describe()

count    132204.000000
mean          0.119283
std          30.127381
min           0.000029
25%           0.008792
50%           0.018116
75%           0.038109
max       10954.299003
Name: likes_views_ratio, dtype: float64

In [101]:
df_videos.sort_values("likes_views_ratio").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio
43093,Classic Mr Bean,UCpVm7bg6pXKo1Pr6k5kxG9A,Official Teaser | Killing Reagan,NDcBhIEwa3U,2016-09-19 14:28:54+00:00,2004634,58,2,3,1871.0,29.0,0.000029
97564,Numberphile,UCoxcjq-8xIDTYp3uz647V5A,Why do YouTube views freeze at 301?,oIkhgagvrjI,2012-06-22 18:38:55+00:00,301,3297244,46478,291405,3421.0,70.942037,10954.299003


## Dislikes/views

In [102]:
df_videos["dislikes_views_ratio"] = df_videos["dislikes"]/df_videos["views"]
df_videos["dislikes_views_ratio"] = df_videos["dislikes_views_ratio"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [103]:
df_videos["dislikes_views_ratio"].describe()

count    1.322040e+05
mean     2.361125e-03
std      4.246873e-01
min      7.519102e-07
25%      2.910347e-04
50%      5.605381e-04
75%      1.097897e-03
max      1.544120e+02
Name: dislikes_views_ratio, dtype: float64

In [104]:
df_videos.sort_values("dislikes_views_ratio").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio,dislikes_views_ratio
58574,Disney Parks,UC1xwwLwm6WSMbUn_Tp597hQ,Europe Family Vacations | Adventures by Disney,BAk5kgg70uo,2019-01-25 14:48:12+00:00,1329946,103,1,1,1013.0,103.0,0.000077,0.000001
97564,Numberphile,UCoxcjq-8xIDTYp3uz647V5A,Why do YouTube views freeze at 301?,oIkhgagvrjI,2012-06-22 18:38:55+00:00,301,3297244,46478,291405,3421.0,70.942037,10954.299003,154.41196


## Comments/views

In [105]:
df_videos["comments_views_ratio"] = df_videos["comments"]/df_videos["views"]
df_videos["comments_views_ratio"] = df_videos["comments_views_ratio"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [106]:
df_videos["comments_views_ratio"].describe()

count    1.322040e+05
mean     1.069282e-02
std      2.662613e+00
min      6.941052e-07
25%      7.824332e-04
50%      1.732919e-03
75%      3.705065e-03
max      9.681229e+02
Name: comments_views_ratio, dtype: float64

In [107]:
df_videos.sort_values("comments_views_ratio").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio,dislikes_views_ratio,comments_views_ratio
69652,Guinness World Records,UCeSRjhfeeqIgr--AcP9qhyg,Cutting the world's longest fingernails #Shorts,GQjeGkuTEg4,2021-08-19 14:32:05+00:00,23051260,560246,48616,16,76.0,11.523902,0.024304,0.002109,0.000001
97564,Numberphile,UCoxcjq-8xIDTYp3uz647V5A,Why do YouTube views freeze at 301?,oIkhgagvrjI,2012-06-22 18:38:55+00:00,301,3297244,46478,291405,3421.0,70.942037,10954.299003,154.41196,968.122924


## Comments/likes

In [108]:
df_videos["comments_likes_ratio"] = df_videos["comments"]/df_videos["likes"]
df_videos["comments_likes_ratio"] = df_videos["comments_likes_ratio"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [109]:
df_videos["comments_likes_ratio"].describe()

count    132204.000000
mean          0.142736
std           0.229970
min           0.000017
25%           0.050711
50%           0.083971
75%           0.152778
max          23.500000
Name: comments_likes_ratio, dtype: float64

In [110]:
df_videos.sort_values("comments_likes_ratio").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio,dislikes_views_ratio,comments_views_ratio,comments_likes_ratio
23311,Buenas Ideias,UCQRPDZMSwXFEDS67uc7kIdg,A GÊNESE DE LULA - EDUARDO BUENO,2myuXzu7IoM,2021-06-16 12:32:31+00:00,465405,57587,5425,1,140.0,10.615115,0.123735,0.011657,0.000002,0.000017
33920,CNBC,UCvJJ_dzjViJCoLf5uKUTwoA,Rep. Steny Hoyer: President Trump's Budget Is ...,tvdHfHf-MKo,2017-03-16 12:57:23+00:00,1040,2,1,47,1693.0,2.0,0.001923,0.000962,0.045192,23.5


## Comments/dislikes

In [111]:
df_videos["comments_dislikes_ratio"] = df_videos["comments"]/df_videos["dislikes"]
df_videos["comments_dislikes_ratio"] = df_videos["comments_dislikes_ratio"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [112]:
df_videos["comments_dislikes_ratio"].describe()

count    132204.000000
mean          5.178932
std           9.757415
min           0.000184
25%           1.466667
50%           3.000000
75%           5.890286
max         726.480769
Name: comments_dislikes_ratio, dtype: float64

In [113]:
df_videos.sort_values("comments_dislikes_ratio").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio,dislikes_views_ratio,comments_views_ratio,comments_likes_ratio,comments_dislikes_ratio
23311,Buenas Ideias,UCQRPDZMSwXFEDS67uc7kIdg,A GÊNESE DE LULA - EDUARDO BUENO,2myuXzu7IoM,2021-06-16 12:32:31+00:00,465405,57587,5425,1,140.0,10.615115,0.123735,0.011657,0.000002,0.000017,0.000184
130434,UrAvgConsumer,UC9fSZHEh6XsRpX-xJc6lT3A,iPhone 6S + LG G4: DOUBLE GIVEAWAY!,RAiQ1MgG9fA,2016-02-09 21:38:37+00:00,105333,9710,52,37777,2094.0,186.730769,0.092184,0.000494,0.358644,3.890525,726.480769


## Mean views per day

In [114]:
df_videos["mean_views_day"] = df_videos["views"]/df_videos["age_days"]
df_videos["mean_views_day"] = df_videos["mean_views_day"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [115]:
df_videos["mean_views_day"].describe()

count    1.322040e+05
mean     9.084332e+02
std      8.886019e+03
min      7.919870e-03
25%      1.251295e+01
50%      6.253708e+01
75%      3.385050e+02
max      1.759142e+06
Name: mean_views_day, dtype: float64

In [116]:
df_videos.sort_values("mean_views_day").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio,dislikes_views_ratio,comments_views_ratio,comments_likes_ratio,comments_dislikes_ratio,mean_views_day
28363,Business Insider,UCcyq283he07B7_KUX07mmtA,From Neutral To Buy- Oil Sector Upgrades,oi3w93LmW3w,2010-02-01 19:13:42+00:00,34,1,1,1,4293.0,1.0,0.029412,0.029412,0.029412,1.0,1.0,0.00792
131267,Veritasium,UCHnyfMqiRRG1u-2MsSQLbXA,How Imaginary Numbers Were Invented,cUzklzVXJwo,2021-11-01 04:53:29+00:00,3518285,229240,1843,11519,2.0,124.384156,0.065157,0.000524,0.003274,0.050249,6.250136,1759142.5


## Mean likes per day

In [117]:
df_videos["mean_likes_day"] = df_videos["likes"]/df_videos["age_days"]
df_videos["mean_likes_day"] = df_videos["mean_likes_day"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [118]:
df_videos["mean_likes_day"].describe()

count    132204.000000
mean         31.259752
std         455.651898
min           0.000227
25%           0.161531
50%           1.291966
75%           9.532371
max      114620.000000
Name: mean_likes_day, dtype: float64

In [119]:
df_videos.sort_values("mean_likes_day").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio,dislikes_views_ratio,comments_views_ratio,comments_likes_ratio,comments_dislikes_ratio,mean_views_day,mean_likes_day
28386,Business Insider,UCcyq283he07B7_KUX07mmtA,Thomas Gensemer On E-Mail During The Obama Cam...,NsrimXL-Pq0,2009-10-02 21:42:20+00:00,124,1,2,2,4415.0,0.5,0.008065,0.016129,0.016129,2.0,1.0,0.028086,0.000227
131267,Veritasium,UCHnyfMqiRRG1u-2MsSQLbXA,How Imaginary Numbers Were Invented,cUzklzVXJwo,2021-11-01 04:53:29+00:00,3518285,229240,1843,11519,2.0,124.384156,0.065157,0.000524,0.003274,0.050249,6.250136,1759142.5,114620.0


## Mean dislikes per day

In [120]:
df_videos["mean_dislikes_day"] = df_videos["dislikes"]/df_videos["age_days"]
df_videos["mean_dislikes_day"] = df_videos["mean_dislikes_day"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [121]:
df_videos["mean_dislikes_day"].describe()

count    132204.000000
mean          0.808790
std          18.155946
min           0.000191
25%           0.006035
50%           0.034483
75%           0.217134
max        5327.174107
Name: mean_dislikes_day, dtype: float64

In [122]:
df_videos.sort_values("mean_dislikes_day").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio,dislikes_views_ratio,comments_views_ratio,comments_likes_ratio,comments_dislikes_ratio,mean_views_day,mean_likes_day,mean_dislikes_day
22570,BBC,UCCj956IF62FbT7Gouszaj9w,Zulu Stick Fighting - Last Man Standing - BBC ...,zCAgEHqZGpI,2007-06-30 02:42:37+00:00,27953,53,1,17,5241.0,53.0,0.001896,0.000036,0.000608,0.320755,17.0,5.333524,0.010113,0.000191
93131,Netflix,UCWOA1ZGywLbqmigxE4Qlvuw,Cuties | Official Trailer | Netflix,M0O7lLe4SmA,2020-08-12 18:38:39+00:00,17866329,94043,2386574,205209,448.0,0.039405,0.005264,0.133579,0.011486,2.182076,0.085985,39880.198661,209.917411,5327.174107


## Mean comments per day

In [123]:
df_videos["mean_comments_day"] = df_videos["comments"]/df_videos["age_days"]
df_videos["mean_comments_day"] = df_videos["mean_comments_day"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [124]:
df_videos["mean_comments_day"].describe()

count    132204.000000
mean          1.984466
std          26.100341
min           0.000192
25%           0.018082
50%           0.116144
75%           0.696633
max        5759.500000
Name: mean_comments_day, dtype: float64

In [125]:
df_videos.sort_values("mean_comments_day").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio,dislikes_views_ratio,comments_views_ratio,comments_likes_ratio,comments_dislikes_ratio,mean_views_day,mean_likes_day,mean_dislikes_day,mean_comments_day
22541,BBC,UCCj956IF62FbT7Gouszaj9w,Adil Ray with Heera - London Mela 2007 - BBC A...,aR8v1FrPBE0,2007-08-12 15:56:57+00:00,6305,7,1,1,5197.0,7.0,0.00111,0.000159,0.000159,0.142857,1.0,1.2132,0.001347,0.000192,0.000192
131267,Veritasium,UCHnyfMqiRRG1u-2MsSQLbXA,How Imaginary Numbers Were Invented,cUzklzVXJwo,2021-11-01 04:53:29+00:00,3518285,229240,1843,11519,2.0,124.384156,0.065157,0.000524,0.003274,0.050249,6.250136,1759142.5,114620.0,921.5,5759.5


## Mean likes/dislikes per day

In [126]:
df_videos["mean_likes_dislikes_ratio_day"] = df_videos["likes_dislikes_ratio"]/df_videos["age_days"]
df_videos["mean_likes_dislikes_ratio_day"] = df_videos["mean_likes_dislikes_ratio_day"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [127]:
df_videos["mean_likes_dislikes_ratio_day"].describe()

count    132204.000000
mean          0.199864
std           2.129878
min           0.000004
25%           0.008415
50%           0.025759
75%           0.077930
max         300.000000
Name: mean_likes_dislikes_ratio_day, dtype: float64

In [128]:
df_videos.sort_values("mean_likes_dislikes_ratio_day").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio,dislikes_views_ratio,comments_views_ratio,comments_likes_ratio,comments_dislikes_ratio,mean_views_day,mean_likes_day,mean_dislikes_day,mean_comments_day,mean_likes_dislikes_ratio_day
46864,Classic Mr Bean,UCpVm7bg6pXKo1Pr6k5kxG9A,American Gypsies | National Geographic,Zkz8F6xexbY,2012-07-02 21:43:01+00:00,267807,201,13309,737,3411.0,0.015103,0.000751,0.049696,0.002752,3.666667,0.055376,78.512753,0.058927,3.901788,0.216066,0.000004
98223,O Pimentinha,UCQ3JxE-NOyZaJ3m3qIkZbhA,O Pimentinha saindo para a oktoberpet,luwzVXWfypc,2021-10-27 17:06:02+00:00,16615,2100,1,186,7.0,2100.0,0.126392,0.00006,0.011195,0.088571,186.0,2373.571429,300.0,0.142857,26.571429,300.0


## Mean likes/views per day

In [129]:
df_videos["mean_likes_views_ratio_day"] = df_videos["likes_views_ratio"]/df_videos["age_days"]
df_videos["mean_likes_views_ratio_day"] = df_videos["mean_likes_views_ratio_day"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [130]:
df_videos["mean_likes_views_ratio_day"].describe()

count    1.322040e+05
mean     1.772831e-04
std      8.923358e-03
min      1.546390e-08
25%      4.170137e-06
50%      1.239631e-05
75%      5.383569e-05
max      3.202075e+00
Name: mean_likes_views_ratio_day, dtype: float64

In [131]:
df_videos.sort_values("mean_likes_views_ratio_day").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio,dislikes_views_ratio,comments_views_ratio,comments_likes_ratio,comments_dislikes_ratio,mean_views_day,mean_likes_day,mean_dislikes_day,mean_comments_day,mean_likes_dislikes_ratio_day,mean_likes_views_ratio_day
43093,Classic Mr Bean,UCpVm7bg6pXKo1Pr6k5kxG9A,Official Teaser | Killing Reagan,NDcBhIEwa3U,2016-09-19 14:28:54+00:00,2004634,58,2,3,1871.0,29.0,0.000029,0.000001,0.000001,0.051724,1.5,1071.423838,0.030999,0.001069,0.001603,0.0155,0.0
97564,Numberphile,UCoxcjq-8xIDTYp3uz647V5A,Why do YouTube views freeze at 301?,oIkhgagvrjI,2012-06-22 18:38:55+00:00,301,3297244,46478,291405,3421.0,70.942037,10954.299003,154.41196,968.122924,0.088378,6.269741,0.087986,963.824613,13.586086,85.181234,0.020737,3.202075


## Mean dislikes/views per day

In [132]:
df_videos["mean_dislikes_views_ratio_day"] = df_videos["dislikes_views_ratio"]/df_videos["age_days"]
df_videos["mean_dislikes_views_ratio_day"] = df_videos["mean_dislikes_views_ratio_day"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [133]:
df_videos["mean_dislikes_views_ratio_day"].describe()

count    1.322040e+05
mean     3.899149e-06
std      1.618690e-04
min      5.332380e-10
25%      1.491122e-07
50%      4.509689e-07
75%      1.371640e-06
max      4.513650e-02
Name: mean_dislikes_views_ratio_day, dtype: float64

In [135]:
df_videos.sort_values("mean_dislikes_views_ratio_day").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio,dislikes_views_ratio,comments_views_ratio,comments_likes_ratio,comments_dislikes_ratio,mean_views_day,mean_likes_day,mean_dislikes_day,mean_comments_day,mean_likes_dislikes_ratio_day,mean_likes_views_ratio_day,mean_dislikes_views_ratio_day
43093,Classic Mr Bean,UCpVm7bg6pXKo1Pr6k5kxG9A,Official Teaser | Killing Reagan,NDcBhIEwa3U,2016-09-19 14:28:54+00:00,2004634,58,2,3,1871.0,29.0,0.000029,0.000001,0.000001,0.051724,1.5,1071.423838,0.030999,0.001069,0.001603,0.0155,0.0,0.0
97564,Numberphile,UCoxcjq-8xIDTYp3uz647V5A,Why do YouTube views freeze at 301?,oIkhgagvrjI,2012-06-22 18:38:55+00:00,301,3297244,46478,291405,3421.0,70.942037,10954.299003,154.41196,968.122924,0.088378,6.269741,0.087986,963.824613,13.586086,85.181234,0.020737,3.202075,0.045136


## Mean comments/views per day

In [136]:
df_videos["mean_comments_views_ratio_day"] = df_videos["comments_views_ratio"]/df_videos["age_days"]
df_videos["mean_comments_views_ratio_day"] = df_videos["mean_comments_views_ratio_day"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [137]:
df_videos["mean_comments_views_ratio_day"].describe()

count    1.322040e+05
mean     1.468426e-05
std      7.918270e-04
min      3.900189e-10
25%      3.879912e-07
50%      1.243654e-06
75%      4.373198e-06
max      2.829941e-01
Name: mean_comments_views_ratio_day, dtype: float64

In [138]:
df_videos.sort_values("mean_comments_views_ratio_day").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio,dislikes_views_ratio,comments_views_ratio,comments_likes_ratio,comments_dislikes_ratio,mean_views_day,mean_likes_day,mean_dislikes_day,mean_comments_day,mean_likes_dislikes_ratio_day,mean_likes_views_ratio_day,mean_dislikes_views_ratio_day,mean_comments_views_ratio_day
59525,Disney Parks,UC1xwwLwm6WSMbUn_Tp597hQ,"Tubestone Curl | Aulani, A Disney Resort & Spa",ZZUZ26lJQ6A,2014-07-31 14:43:45+00:00,2900428,206,18,3,2652.0,11.444444,0.000071,0.000006,0.000001,0.014563,0.166667,1093.675716,0.077677,0.006787,0.001131,0.004315,0.0,0.0,0.0
97564,Numberphile,UCoxcjq-8xIDTYp3uz647V5A,Why do YouTube views freeze at 301?,oIkhgagvrjI,2012-06-22 18:38:55+00:00,301,3297244,46478,291405,3421.0,70.942037,10954.299003,154.41196,968.122924,0.088378,6.269741,0.087986,963.824613,13.586086,85.181234,0.020737,3.202075,0.045136,0.282994


## Mean comments/likes per day

In [139]:
df_videos["mean_comments_likes_ratio_day"] = df_videos["comments_likes_ratio"]/df_videos["age_days"]
df_videos["mean_comments_likes_ratio_day"] = df_videos["mean_comments_likes_ratio_day"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [140]:
df_videos["mean_comments_likes_ratio_day"].describe()

count    1.322040e+05
mean     2.634230e-04
std      3.425050e-03
min      1.240359e-07
25%      3.238680e-05
50%      6.350243e-05
75%      1.431594e-04
max      8.011494e-01
Name: mean_comments_likes_ratio_day, dtype: float64

In [141]:
df_videos.sort_values("mean_comments_likes_ratio_day").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio,dislikes_views_ratio,comments_views_ratio,comments_likes_ratio,comments_dislikes_ratio,mean_views_day,mean_likes_day,mean_dislikes_day,mean_comments_day,mean_likes_dislikes_ratio_day,mean_likes_views_ratio_day,mean_dislikes_views_ratio_day,mean_comments_views_ratio_day,mean_comments_likes_ratio_day
23311,Buenas Ideias,UCQRPDZMSwXFEDS67uc7kIdg,A GÊNESE DE LULA - EDUARDO BUENO,2myuXzu7IoM,2021-06-16 12:32:31+00:00,465405,57587,5425,1,140.0,10.615115,0.123735,0.011657,0.000002,0.000017,0.000184,3324.321429,411.335714,38.75,0.007143,0.075822,0.000884,0.000083,0.0,0.0
9471,BBC,UCCj956IF62FbT7Gouszaj9w,"Climate at one minute to midnight, Boris Johns...",gWqrD-Xl6bQ,2021-11-01 06:58:56+00:00,134143,870,1017,1394,2.0,0.855457,0.006486,0.007581,0.010392,1.602299,1.370698,67071.5,435.0,508.5,697.0,0.427729,0.003243,0.003791,0.005196,0.801149


## Mean comments/dislikes per day

In [143]:
df_videos["mean_comments_dislikes_ratio_day"] = df_videos["comments_dislikes_ratio"]/df_videos["age_days"]
df_videos["mean_comments_dislikes_ratio_day"] = df_videos["mean_comments_dislikes_ratio_day"].replace([np.inf, -np.inf], np.nan)
df_videos = df_videos.dropna()
df_videos = df_videos.reset_index(drop = True)

In [144]:
df_videos["mean_comments_dislikes_ratio_day"].describe()

count    132204.000000
mean          0.014821
std           0.175508
min           0.000001
25%           0.000813
50%           0.002091
75%           0.005955
max          26.571429
Name: mean_comments_dislikes_ratio_day, dtype: float64

In [145]:
df_videos.sort_values("mean_comments_dislikes_ratio_day").apply(lambda x: pd.concat([x.head(1), x.tail(1)]))

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio,dislikes_views_ratio,comments_views_ratio,comments_likes_ratio,comments_dislikes_ratio,mean_views_day,mean_likes_day,mean_dislikes_day,mean_comments_day,mean_likes_dislikes_ratio_day,mean_likes_views_ratio_day,mean_dislikes_views_ratio_day,mean_comments_views_ratio_day,mean_comments_likes_ratio_day,mean_comments_dislikes_ratio_day
23311,Buenas Ideias,UCQRPDZMSwXFEDS67uc7kIdg,A GÊNESE DE LULA - EDUARDO BUENO,2myuXzu7IoM,2021-06-16 12:32:31+00:00,465405,57587,5425,1,140.0,10.615115,0.123735,0.011657,0.000002,0.000017,0.000184,3324.321429,411.335714,38.75,0.007143,0.075822,0.000884,0.000083,0.0,0.0,0.000001
98223,O Pimentinha,UCQ3JxE-NOyZaJ3m3qIkZbhA,O Pimentinha saindo para a oktoberpet,luwzVXWfypc,2021-10-27 17:06:02+00:00,16615,2100,1,186,7.0,2100.0,0.126392,0.00006,0.011195,0.088571,186.0,2373.571429,300.0,0.142857,26.571429,300.0,0.018056,0.000009,0.001599,0.012653,26.571429


In [146]:
df_videos

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments,age_days,likes_dislikes_ratio,likes_views_ratio,dislikes_views_ratio,comments_views_ratio,comments_likes_ratio,comments_dislikes_ratio,mean_views_day,mean_likes_day,mean_dislikes_day,mean_comments_day,mean_likes_dislikes_ratio_day,mean_likes_views_ratio_day,mean_dislikes_views_ratio_day,mean_comments_views_ratio_day,mean_comments_likes_ratio_day,mean_comments_dislikes_ratio_day
0,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,A few of the best math explainers from this su...,F3Qixy-r_rQ,2021-10-23 18:11:23+00:00,502452,29457,107,768,11.0,275.299065,0.058626,0.000213,0.001529,0.026072,7.17757,45677.454545,2677.909091,9.727273,69.818182,25.027188,0.00533,0.000019,0.000139,0.00237,0.652506
1,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,How a Mandelbrot set arises from Newton’s work,LqbZpur38nw,2021-10-15 16:41:50+00:00,600077,26908,150,1237,19.0,179.386667,0.044841,0.00025,0.002061,0.045971,8.246667,31583.0,1416.210526,7.894737,65.105263,9.441404,0.00236,0.000013,0.000108,0.00242,0.434035
2,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,Newton's Fractal (which Newton knew nothing ab...,-RdOwhmqP5s,2021-10-07 02:19:39+00:00,1235429,62901,294,2850,28.0,213.94898,0.050914,0.000238,0.002307,0.045309,9.693878,44122.464286,2246.464286,10.5,101.785714,7.641035,0.001818,0.000008,0.000082,0.001618,0.34621
3,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,The Summer of Math Exposition,ojjzXyQCzso,2021-07-16 15:37:16+00:00,613493,29362,215,1722,110.0,136.567442,0.04786,0.00035,0.002807,0.058647,8.009302,5577.209091,266.927273,1.954545,15.654545,1.241522,0.000435,0.000003,0.000026,0.000533,0.072812
4,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,A quick trick for computing eigenvalues | Chap...,e50Bj7jn9IQ,2021-05-07 19:01:16+00:00,427767,17155,145,1150,180.0,118.310345,0.040104,0.000339,0.002688,0.067036,7.931034,2376.483333,95.305556,0.805556,6.388889,0.65728,0.000223,0.000002,0.000015,0.000372,0.044061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132199,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Art Hour,C6hpMftAIs0,2009-06-13 18:32:12+00:00,172809,2160,31,136,4526.0,69.677419,0.012499,0.000179,0.000787,0.062963,4.387097,38.181396,0.477243,0.006849,0.030049,0.015395,0.000003,0.0,0.0,0.000014,0.000969
132200,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Hole,LPfcGXMpKds,2009-06-06 22:16:56+00:00,90894,1799,23,75,4533.0,78.217391,0.019792,0.000253,0.000825,0.04169,3.26087,20.051621,0.396867,0.005074,0.016545,0.017255,0.000004,0.0,0.0,0.000009,0.000719
132201,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Affordable Button,h3lvSflNixI,2009-05-30 00:13:46+00:00,103229,1570,47,18,4541.0,33.404255,0.015209,0.000455,0.000174,0.011465,0.382979,22.732658,0.345739,0.01035,0.003964,0.007356,0.000003,0.0,0.0,0.000003,0.000084
132202,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Outsource,zxIbQ6ZG4OI,2009-05-17 00:50:31+00:00,78267,1625,20,87,4554.0,81.25,0.020762,0.000256,0.001112,0.053538,4.35,17.18643,0.356829,0.004392,0.019104,0.017841,0.000005,0.0,0.0,0.000012,0.000955


# Save the dataset

In [147]:
df_videos.to_csv("data/videos_data.csv",
                 sep = ";",
                 index = False)